In [1]:
import numpy as np
import pandas as pd
import warnings
import os
import sys
import math
import copy
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)


class UserCF:
    def __init__(self):
        pass
    
    def load_data(self):
        path1 = 'movielens_data/users.dat'
        path2 = 'movielens_data/ratings.dat'
        path3 = 'movielens_data/movies.dat'
        
        users = pd.read_csv(path1, header = None, sep = '::')
        ratings = pd.read_csv(path2, header = None, sep = '::')
        movies = pd.read_csv(path3, header = None, sep = '::')
        
        users.columns = ['user_id', 'gender', 'age', 'occupation', 'zip_code']
        ratings.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
        movies.columns = ['movie_id', 'title', 'genres']
        
        return users, ratings, movies
    
    def split_train_test(self, users, ratings):
        all_data = ratings.merge(users, on = 'user_id', how = 'left')
        
        shuffled_indices = np.random.permutation(len(all_data))
        test_set_size = int(len(all_data) * 0.2)
        
        test_indices = shuffled_indices[:test_set_size]
        train_indices = shuffled_indices[test_set_size:]
        
        return all_data.iloc[train_indices], all_data.iloc[test_indices]
    
    def user_similarity(self, train_data):
        
        # build inverse table for item_users
        print('establising the inverse table...')
        self.item_users = dict()
        for index, row in train_data.iterrows():
            if row['movie_id'] not in self.item_users:
                self.item_users[row['movie_id']] = set()
            self.item_users[row['movie_id']].add(row['user_id'])
            
        # calculate co-rated items between users
        print('calculating the co-rated matrix...')
        self.user_list = list(set(train_data['user_id']))
        movie_list_len = len(set(train_data['movie_id']))
        
        C = [[0 for i in range(len(self.user_list))] for i in range(len(self.user_list))]
        W = copy.deepcopy(C)
        
        self.N = dict()
        i = 1
        for movie, users in self.item_users.items():
            #print(str(i) + ' in ' + str(movie_list_len))
            i += 1
            for u in users:
                if self.user_list.index(u) not in self.N:
                    self.N[self.user_list.index(u)] = 0
                self.N[self.user_list.index(u)] += 1
                for v in users:
                    if u == v:
                        continue
                    #C[self.user_list.index(u)][self.user_list.index(v)] += 1
                    C[self.user_list.index(u)][self.user_list.index(v)] += 1 / math.log(1 + len(users))
                    
        # calculate finial similarity matrix W
        print('calculating the final similarity matrix...')
        for i in range(len(C)):
            for j in range(len(C)):
                if i == j or C[i][j] == 0:
                    continue
                W[i][j] = C[i][j] / math.sqrt(self.N[i] * self.N[j])
        
        return W
    
    def recommend(self, ratings, user, W, k=5, threshold=3):
        # find the first k highest similarity values
        similar = sorted(enumerate(W[self.user_list.index(user)]), reverse=True, key=lambda x:x[1])[0:k]
        
        movie_list = np.array(k)
        first_k = dict()
        for s in similar:
            # {user_index : similarity_value}
            u_id = self.user_list[s[0]]
            first_k[u_id] = s[1]
            # find the movies which the similar uses have interacted
            movie_list = np.append(movie_list, np.array(ratings.loc[ratings['user_id'] == s[0]]['movie_id'].values))
        
        # if the highest value is zero, retun emtpy
        if similar[0][1] == 0:
            return []
    
    
        recommend_score = dict()
    
        # if users's 'similar users' have touched the product,
        # but the user have interacted with the producet,
        # give an evluation more than the threshold,
        # then recommend it to the current user
        for movie in movie_list:
            if movie in self.item_users:
                if user not in self.item_users[movie]:    # if user didn't interact with the product
                    for u in first_k.keys():
                        if u in self.item_users[movie]:    # if the similar users have interacted with the product
                            rating = int(ratings.loc[(ratings['user_id'] == u) & (ratings['movie_id'] == movie)]['rating'])
                            if rating >= threshold:
                                if movie not in recommend_score:
                                    recommend_score[movie] = 0
                                recommend_score[movie] += first_k[u] * rating     

                        
        return recommend_score

In [2]:
users, ratings, movies = UserCF().load_data()
print('data loaded')

data loaded


In [3]:
ucf = UserCF()
train_data, test_data = ucf.split_train_test(users, ratings)
W = ucf.user_similarity(train_data.head(int(train_data.shape[0]/50)))
print('done')

establising the inverse table...
calculating the co-rated matrix...
calculating the final similarity matrix...
done


In [5]:
test_user = list(set(test_data['user_id']))
for user in test_user:
    if user not in ucf.user_list:
        continue
    result = ucf.recommend(ratings, user, W, k=10, threshold=5)
    if len(result) != 0:
        print('user_id: ', user)
        for movie, score in result.items():
            print(movie, movies.loc[movies['movie_id'] == movie]['title'].values[0], score)
        print('==================================================')

user_id:  2
940 Adventures of Robin Hood, The (1938) 0.5718999268501628
user_id:  3
590 Dances with Wolves (1990) 0
1197 Princess Bride, The (1987) 0
1256 Duck Soup (1933) 1.5533373363990297
3809 What About Bob? (1991) 0
1387 Jaws (1975) 0
1192 Paris Is Burning (1990) 0
user_id:  5
3408 Erin Brockovich (2000) 1.6042799026441679
1198 Raiders of the Lost Ark (1981) 2.690459187936828
924 2001: A Space Odyssey (1968) 0.8021399513220839
user_id:  8
1219 Psycho (1960) 0.795821346874263
1252 Chinatown (1974) 1.3784030065341186
1079 Fish Called Wanda, A (1988) 0.795821346874263
user_id:  9
1288 This Is Spinal Tap (1984) 1.3901079714141542
296 Pulp Fiction (1994) 4.170323914242463
user_id:  10
1214 Alien (1979) 1.3893471504706754
1222 Full Metal Jacket (1987) 0.45946766884470625
2716 Ghostbusters (1984) 1.8524628672942338
user_id:  11
589 Terminator 2: Judgment Day (1991) 5.294341857971418
user_id:  13
3753 Patriot, The (2000) 2.9697420437337008
1617 L.A. Confidential (1997) 1.8718325148456874


user_id:  70
1197 Princess Bride, The (1987) 0
590 Dances with Wolves (1990) 0
3809 What About Bob? (1991) 0
1387 Jaws (1975) 0
47 Seven (Se7en) (1995) 1.0441160008718422
1192 Paris Is Burning (1990) 0
1639 Chasing Amy (1997) 1.0441160008718422
user_id:  72
1653 Gattaca (1997) 2.306508375074624
3104 Midnight Run (1988) 2.1186632273957953
1923 There's Something About Mary (1998) 3.459762562611936
1079 Fish Called Wanda, A (1988) 2.1186632273957953
553 Tombstone (1993) 1.729881281305968
1947 West Side Story (1961) 1.5472528552018323
3262 Twin Peaks: Fire Walk with Me (1992) 0.521578831086467
2167 Blade (1998) 1.153254187537312
3819 Tampopo (1986) 0.9987474234280955
user_id:  75
1291 Indiana Jones and the Last Crusade (1989) 2.7937669335094917
2478 Three Amigos! (1986) 0.9312556445031639
3809 What About Bob? (1991) 0
1197 Princess Bride, The (1987) 0
1387 Jaws (1975) 0
1200 Aliens (1986) 1.8625112890063278
590 Dances with Wolves (1990) 0
608 Fargo (1996) 2.7937669335094917
1374 Star Trek:

user_id:  129
110 Braveheart (1995) 2.599141635008534
2716 Ghostbusters (1984) 2.9471495950570255
1358 Sling Blade (1996) 1.0813082812172974
2762 Sixth Sense, The (1999) 3.4460075163352966
908 North by Northwest (1959) 2.9240343393846007
1198 Raiders of the Lost Ark (1981) 1.3784030065341186
2291 Edward Scissorhands (1990) 1.1254613419756059
3668 Romeo and Juliet (1968) 0.9746781131282002
16 Casino (1995) 1.0813082812172974
user_id:  130
1198 Raiders of the Lost Ark (1981) 8.399698949995583
user_id:  131
919 Wizard of Oz, The (1939) 0.7677314329642192
user_id:  132
10 GoldenEye (1995) 1.0441160008718422
590 Dances with Wolves (1990) 0
1387 Jaws (1975) 1.3138171562999033
2700 South Park: Bigger, Longer and Uncut (1999) 1.7201904480216956
2571 Matrix, The (1999) 0.8758781041999354
2353 Enemy of the State (1998) 0.8600952240108478
user_id:  133
1307 When Harry Met Sally... (1989) 2.261303854140338
1090 Platoon (1986) 1.2308979102295168
user_id:  135
1263 Deer Hunter, The (1978) 1.83416021

user_id:  187
3175 Galaxy Quest (1999) 2.690459187936828
2716 Ghostbusters (1984) 1.7613189832728477
2067 Doctor Zhivago (1965) 0.6946735752353377
user_id:  188
1247 Graduate, The (1967) 2.0591421548193076
user_id:  190
1234 Sting, The (1973) 1.4409373417654023
1213 GoodFellas (1990) 0.8055633011923833
user_id:  192
2683 Austin Powers: The Spy Who Shagged Me (1999) 2.1714724095162588
1387 Jaws (1975) 1.3882339503596355
1663 Stripes (1981) 0.6941169751798177
316 Stargate (1994) 0.6941169751798177
62 Mr. Holland's Opus (1995) 0.6941169751798177
910 Some Like It Hot (1959) 0.8021399513220839
user_id:  194
2762 Sixth Sense, The (1999) 5.4507150409526535
2194 Untouchables, The (1987) 1.4744321597526358
1206 Clockwork Orange, A (1971) 1.6718489227462852
2112 Grand Canyon (1991) 0.6867255459923792
2359 Waking Ned Devine (1998) 1.5638714678409236
3501 Murphy's Romance (1985) 0.5572829742487617
3545 Cabaret (1972) 0.5572829742487617
user_id:  195
3408 Erin Brockovich (2000) 0.6113668398618232
u

user_id:  258
2858 American Beauty (1999) 4.281237746306933
47 Seven (Se7en) (1995) 1.412442151597197
3095 Grapes of Wrath, The (1940) 1.412442151597197
user_id:  259
1213 GoodFellas (1990) 1.9632592803453743
912 Casablanca (1942) 0.9816296401726872
user_id:  261
708 Truth About Cats & Dogs, The (1996) 0.39120561964093464
1214 Alien (1979) 1.5648224785637386
318 Shawshank Redemption, The (1994) 2.2811011491194377
342 Muriel's Wedding (1994) 0.39120561964093464
1246 Dead Poets Society (1989) 1.0739830855871932
231 Dumb & Dumber (1994) 0.39464380868665966
user_id:  262
589 Terminator 2: Judgment Day (1991) 4.783934833557015
2273 Rush Hour (1998) 0.6510110724967255
user_id:  263
3753 Patriot, The (2000) 3.1498871062483444
908 North by Northwest (1959) 3.1498871062483444
user_id:  264
2019 Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954) 0.6239866499667939
1278 Young Frankenstein (1974) 1.8719599499003816
1101 Top Gun (1986) 1.2479732999335877
2248 Say Anything... (1989)

...

user_id:  1785
1248 Touch of Evil (1958) 1.1677322890400232
1183 English Patient, The (1996) 1.1677322890400232
32 Twelve Monkeys (1995) 1.1677322890400232
2011 Back to the Future Part II (1989) 0.7537679513801127
1197 Princess Bride, The (1987) 1.5075359027602253
2064 Roger & Me (1989) 0.5838661445200116
2109 Jerk, The (1979) 0.7537679513801127
user_id:  1788
1097 E.T. the Extra-Terrestrial (1982) 6.765505523125405
593 Silence of the Lambs, The (1991) 3.3827527615627027
608 Fargo (1996) 3.3827527615627027
user_id:  1789
50 Usual Suspects, The (1995) 1.9811194602988127
user_id:  1790
1247 Graduate, The (1967) 1.0295710774096538
user_id:  1791
2028 Saving Private Ryan (1998) 5.939484087467401
3869 Naked Gun 2 1/2: The Smell of Fear, The (1991) 1.0857362047581294
user_id:  1792
590 Dances with Wolves (1990) 0
1387 Jaws (1975) 0
2571 Matrix, The (1999) 1.3138171562999033
2692 Run Lola Run (Lola rennt) (1998) 0.5363636080415295
user_id:  1793
3526 Parenthood (1989) 0.6577396811444329
user_

user_id:  1847
593 Silence of the Lambs, The (1991) 1.5346383821506744
296 Pulp Fiction (1994) 1.5346383821506744
2788 And Now for Something Completely Different (1971) 0.5115461273835581
user_id:  1850
1196 Star Wars: Episode V - The Empire Strikes Back (1980) 5.043280696178908
367 Mask, The (1994) 2.1614060126481034
913 Maltese Falcon, The (1941) 1.4409373417654023
user_id:  1853
594 Snow White and the Seven Dwarfs (1937) 0.6400855688321548
750 Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963) 1.9202567064964646
3152 Last Picture Show, The (1971) 0.6268500900860287
1220 Blues Brothers, The (1980) 1.456764871155737
910 Some Like It Hot (1959) 1.9202567064964646
2010 Metropolis (1926) 0.6400855688321548
user_id:  1854
2194 Untouchables, The (1987) 1.8033688011112043
user_id:  1855
1380 Grease (1978) 4.598573236878717
2054 Honey, I Shrunk the Kids (1989) 0.8214559901046841
3753 Patriot, The (2000) 3.5570087776077735
user_id:  1856
1997 Exorcist, The (1973) 1.61

user_id:  1906
1302 Field of Dreams (1989) 0.8302937312698839
1197 Princess Bride, The (1987) 0
1387 Jaws (1975) 0
2571 Matrix, The (1999) 2.3915192781660695
590 Dances with Wolves (1990) 0
3809 What About Bob? (1991) 0
1192 Paris Is Burning (1990) 0
user_id:  1908
590 Dances with Wolves (1990) 0
908 North by Northwest (1959) 1.072727216083059
260 Star Wars: Episode IV - A New Hope (1977) 3.0056146685186738
318 Shawshank Redemption, The (1994) 2.092914849567177
356 Forrest Gump (1994) 1.547920370316812
34 Babe (1995) 3.222253204769533
1641 Full Monty, The (1997) 0.5363636080415295
2858 American Beauty (1999) 3.0056146685186738
1387 Jaws (1975) 0
2159 Henry: Portrait of a Serial Killer (1990) 0.386980092579203
3526 Parenthood (1989) 1.072727216083059
1042 That Thing You Do! (1996) 0.5363636080415295
user_id:  1910
1265 Groundhog Day (1993) 3.85423756777313
590 Dances with Wolves (1990) 0
1387 Jaws (1975) 0
1079 Fish Called Wanda, A (1988) 1.2847458559243767
user_id:  1916
3501 Murphy's 

user_id:  1975
1719 Sweet Hereafter, The (1997) 2.1094598884529177
1213 GoodFellas (1990) 1.8268458516645152
1089 Reservoir Dogs (1992) 2.1094598884529177
590 Dances with Wolves (1990) 0
1387 Jaws (1975) 0
39 Clueless (1995) 0.6089486172215051
1192 Paris Is Burning (1990) 0
user_id:  1977
908 North by Northwest (1959) 2.261303854140338
1968 Breakfast Club, The (1985) 1.8463468653442752
1674 Witness (1985) 1.1779555682072247
user_id:  1978
2302 My Cousin Vinny (1992) 1.2022458674074696
3175 Galaxy Quest (1999) 1.2022458674074696
1387 Jaws (1975) 0
356 Forrest Gump (1994) 2.776467900719271
720 Wallace & Gromit: The Best of Aardman Animation (1996) 0.6011229337037348
590 Dances with Wolves (1990) 0
1198 Raiders of the Lost Ark (1981) 1.4499630719803436
1527 Fifth Element, The (1997) 0.6668860185618942
user_id:  1982
293 Professional, The (a.k.a. Leon: The Professional) (1994) 0.5363636080415295
318 Shawshank Redemption, The (1994) 1.6429119802093681
1197 Princess Bride, The (1987) 2.62763

KeyboardInterrupt: 

In [88]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [133]:
a = [0,1,2]
b = [a] * len(a)
for i in range(len(b)):
    for j in range(len(b)):
        if i == j:
            continue
        b[i][j] += 3
print(id(b[0]), id(b[1]))

140478493192648 140478493192648


In [131]:
a = [0,1,2]
b = [[0 for i in range(len(a))] for i in range(len(a))]
for i in range(len(b)):
    for j in range(len(b)):
        if i == j:
            continue
        b[i][j] += 3
b

[[0, 3, 3], [3, 0, 3], [3, 3, 0]]